<a href="https://colab.research.google.com/github/ritik-hue/Cap-X/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 7.3 MB/s eta 0:00:00


In [3]:
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.9 MB/s eta 0:00:00


In [8]:
import praw
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import yfinance as yf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from keras.models import Sequential
from keras.layers import Dense, LSTM
import numpy as np
import nltk

nltk.download('stopwords')
nltk.download('punkt_tab')

#Web Scraping

reddit = praw.Reddit(
    client_id="wvOPoMLwWmOLLPiA21g01g",
    client_secret="IU8WZOVsS3_3nRO6UlmR6R3yUZAeow",
    user_agent="python:web scrap3er:1.0 (by /u/pessimistic_singh12)"
)

# Sentiment Analysis and Key Feature Extraction
stop_words = set(stopwords.words("english"))
analyzer = SentimentIntensityAnalyzer()

# Fetching posts from Reddit
subreddit = reddit.subreddit("wallstreetbets")
posts = subreddit.search("AAPL", limit=50)

# Extracting sentiment data
social_media_data = []
for post in posts:
    title = post.title
    body = post.selftext
    post_date = pd.to_datetime(post.created_utc, unit='s').date()

    title_tokens = word_tokenize(title)
    filtered_title = [word for word in title_tokens if word.lower() not in stop_words]

    body_tokens = word_tokenize(body)
    filtered_body = [word for word in body_tokens if word.lower() not in stop_words]

    combined_text = ' '.join(filtered_title) + ' ' + ' '.join(filtered_body)
    sentiment = analyzer.polarity_scores(combined_text)

    social_media_data.append({
        'date': post_date,
        'sentiment': sentiment['compound']
    })

social_media_df = pd.DataFrame(social_media_data)

# Fetching stock data
stock_data = yf.download("AAPL", start="2022-01-01", end="2022-12-31")
stock_data['Date'] = stock_data.index.date
stock_data['Date'] = pd.to_datetime(stock_data['Date'])
stock_data['Daily Return'] = stock_data['Adj Close'].pct_change()
stock_data['Volatility'] = stock_data['Daily Return'].rolling(window=21).std()
stock_data['Moving Average'] = stock_data['Adj Close'].rolling(window=21).mean()

# Preparing data for models
X_stock = stock_data[['Daily Return', 'Volatility', 'Moving Average']]
y_stock = stock_data['Adj Close']

X_train_stock, X_test_stock, y_train_stock, y_test_stock = train_test_split(
    X_stock, y_stock, test_size=0.2, random_state=42
)

# XGBoost Regression
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X_train_stock, y_train_stock)
y_pred_xgb = xgb_model.predict(X_test_stock)
print("XGBoost Regression MSE:", mean_squared_error(y_test_stock, y_pred_xgb))

# LSTM Regression
y_train_stock = y_train_stock.dropna()
y_test_stock = y_test_stock.dropna()
X_stock_lstm = X_train_stock.to_numpy().reshape(X_train_stock.shape[0], X_train_stock.shape[1], 1)
X_test_lstm = X_test_stock.to_numpy().reshape(X_test_stock.shape[0], X_test_stock.shape[1], 1)

lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=False, input_shape=(X_stock_lstm.shape[1], 1)))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

lstm_model.fit(X_stock_lstm, y_train_stock, epochs=10, batch_size=32)

y_pred_lstm = lstm_model.predict(X_test_lstm)
y_pred_lstm = np.nan_to_num(y_pred_lstm)
y_test_stock = y_test_stock.iloc[:len(y_pred_lstm)]

print("LSTM Regression MSE:", mean_squared_error(y_test_stock, y_pred_lstm))
# Processing sentiment data for classification (up/down/neutral)
social_media_df['sentiment_class'] = social_media_df['sentiment'].apply(lambda x: 'positive' if x > 0 else ('negative' if x < 0 else 'neutral'))

# Splitting sentiment data into train, validation, and test sets for classification
X_sentiment = social_media_df[['sentiment']]
y_sentiment = social_media_df['sentiment_class']

X_train_sentiment, X_test_sentiment, y_train_sentiment, y_test_sentiment = train_test_split(X_sentiment, y_sentiment, test_size=0.2, random_state=42)

# Logistic Regression for Sentiment Classification
from sklearn.linear_model import LogisticRegression
log_reg_model = LogisticRegression(max_iter=1000)
log_reg_model.fit(X_train_sentiment, y_train_sentiment)

y_pred_sentiment = log_reg_model.predict(X_test_sentiment)

# Evaluate Logistic Regression Model for Sentiment Classification
from sklearn.metrics import accuracy_score
log_reg_accuracy = accuracy_score(y_test_sentiment, y_pred_sentiment)
print("Logistic Regression Sentiment Classification Accuracy:", log_reg_accuracy)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


XGBoost Regression MSE: 27.332673203443203
Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: nan
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: nan 
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: nan 
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: nan 
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan  
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: nan  
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: nan 
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: nan 
Epoch 9/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: nan 
Epoch 10/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: nan 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
LSTM Regression MSE: 23629.089973988022
Logistic Regression Sentiment Classification Accuracy: 0.8
